In [3]:
import numpy as np
import skimage as sk
import skimage.io as skio
import cv2

In [4]:
def return_score(img_1, img_2, h_dis, w_dis):


    h, w = img_1.shape
    w_start = max(0, w_dis)
    w_end = min(w, w+w_dis)
    h_start = max(0, h_dis)
    h_end = min(h, h+h_dis)

    img_1_cropped = img_1[h_start:h_end, w_start:w_end].copy()
    img_2_cropped = img_2[h_start-h_dis:h_end-h_dis, w_start-w_dis:w_end-w_dis].copy()

    return np.sqrt(np.sum((img_1_cropped - img_2_cropped) ** 2))


def return_score_ncc(img_1, img_2, h_dis, w_dis):


    h, w = img_1.shape
    w_start = max(0, w_dis)
    w_end = min(w, w+w_dis)
    h_start = max(0, h_dis)
    h_end = min(h, h+h_dis)

    img_1_cropped = img_1[h_start:h_end, w_start:w_end].copy()
    img_2_cropped = img_2[h_start-h_dis:h_end-h_dis, w_start-w_dis:w_end-w_dis].copy()
    crop_h = int(0.05 * h)
    crop_w = int(0.05 * w)
    img_1_cropped = img_1_cropped[crop_h: -crop_h, crop_w: -crop_w]
    img_2_cropped = img_2_cropped[crop_h: -crop_h, crop_w: -crop_w]

    img_1_mean = img_1_cropped - np.mean(img_1_cropped)
    img_2_mean = img_2_cropped - np.mean(img_2_cropped)

    numerator = np.sum(img_1_mean * img_2_mean)
    denominator = np.sqrt(np.sum(img_1_mean ** 2) * np.sum(img_2_mean ** 2))

    if denominator == 0:
        return 0  

    return -1 * numerator / denominator


In [5]:
def small_image(name):
    # name of the input file
    imname = f'{name}.jpg'

    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)
        
    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(np.int64)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    # crop images
    h, w = b.shape
    crop_h = int(0.05 * h)
    crop_w = int(0.05 * w)
    b = b[crop_h: -crop_h, crop_w: -crop_w]
    g = g[crop_h: -crop_h, crop_w: -crop_w]
    r = r[crop_h: -crop_h, crop_w: -crop_w]

    smallest_score = float("inf")
    # find b, r displacement
    for i in range(-15, 16):
        for j in range(-15, 16):
            score = return_score(b, r, i, j)
            if score < smallest_score:
                smallest_score = score
                best_w = i
                best_h = j
    b_r_dis = (best_h, best_w)

    smallest_score = float("inf")
    # find b, g displacement
    for i in range(-15, 16):
        for j in range(-15, 16):
            score = return_score(b, g, i, j)
            if score < smallest_score:
                smallest_score = score
                best_w = i
                best_h = j
    b_g_dis = (best_h, best_w)

    r = np.roll(r, b_r_dis[1], axis = 0)
    r = np.roll(r, b_r_dis[0], axis = 1)
    g = np.roll(g, b_g_dis[1], axis = 0)
    g = np.roll(g, b_g_dis[0], axis = 1)

    new_stacked = np.dstack([r, g, b])
    new_stacked = np.clip(new_stacked, 0, 1)
    new_stacked = (new_stacked * 255).astype(np.uint8)
    print("b r displacement", b_r_dis)
    print("b g displacement", b_g_dis)

    # save the image
    fname = f'{name}_final.jpg'
    skio.imsave(fname, new_stacked)

In [6]:
small_image("cathedral")

b r displacement (3, 12)
b g displacement (2, 5)


In [7]:
small_image("tobolsk")

b r displacement (3, 6)
b g displacement (3, 3)


In [8]:
small_image("monastery")

b r displacement (2, 3)
b g displacement (2, -3)


In [4]:
def resize_image(img, factor):
    img = img.astype(np.float32)

    resized_image = cv2.resize(img, (0, 0), fx=1/factor, fy=1/factor, interpolation=cv2.INTER_LINEAR)
    return resized_image

In [105]:
def image_pyramid(name):
    # name of the input file
    imname = f'{name}.tif'

    # read in the image
    im = skio.imread(imname)


    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)
        
    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(np.int64)

    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]

    b_mean = np.mean(b)
    g_mean = np.mean(g)
    r_mean = np.mean(r)


    g = g + (b_mean - g_mean)
    r = r + (b_mean - r_mean)

    # crop images
    h, w = b.shape
    crop_h = int(0.05 * h)
    crop_w = int(0.05 * w)
    b = b[crop_h: -crop_h, crop_w: -crop_w]
    g = g[crop_h: -crop_h, crop_w: -crop_w]
    r = r[crop_h: -crop_h, crop_w: -crop_w]


    b_g_dis = (0, 0)
    b_r_dis = (0, 0)
    final_b_g_dis = (0, 0)
    final_b_r_dis = (0, 0)

    factors = [16, 8, 4, 2, 1]
    window_sizes = [10, 10, 8, 6, 6]
    for i in range(5):
        factor = factors[i]
        window = window_sizes[i]
        final_b_r_dis = (final_b_r_dis[0] + b_r_dis[0] * factor * 2, final_b_r_dis[1] + b_r_dis[1] * factor * 2)
        final_b_g_dis = (final_b_g_dis[0] + b_g_dis[0] * factor * 2, final_b_g_dis[1] + b_g_dis[1] * factor * 2)
        r = np.roll(r, b_r_dis[1] * factor * 2, axis = 0)
        r = np.roll(r, b_r_dis[0] * factor * 2, axis = 1)
        g = np.roll(g, b_g_dis[1] * factor * 2, axis = 0)
        g = np.roll(g, b_g_dis[0] * factor * 2, axis = 1)

        new_b = resize_image(b, factor)
        new_g = resize_image(g, factor)
        new_r = resize_image(r, factor)

        smallest_score = float("inf")
        # find b, g displacement
        for i in range(-window, window):
            for j in range(-window, window):
                score = return_score_ncc(new_b, new_g, i, j)
                if score < smallest_score:
                    smallest_score = score
                    best_w = i
                    best_h = j
        b_g_dis = (best_h, best_w)

        smallest_score = float("inf")
        # find b, r displacement
        for i in range(-window, window):
            for j in range(-window, window):
                score = return_score_ncc(new_b, new_r, i, j)
                if score < smallest_score:
                    smallest_score = score
                    best_w = i
                    best_h = j
        b_r_dis = (best_h, best_w)

        

    g = g - (b_mean - g_mean)
    r = r - (b_mean - r_mean)
    r = np.roll(r, b_r_dis[0], axis = 0)
    r = np.roll(r, b_r_dis[1], axis = 1)
    g = np.roll(g, b_g_dis[0], axis = 0)
    g = np.roll(g, b_g_dis[1], axis = 1)
    print("b r displacement", final_b_r_dis)
    print("b g displacement", final_b_g_dis)

    new_stacked = np.dstack([r, g, b])
    new_stacked = np.clip(new_stacked, 0, 1)
    new_stacked = (new_stacked * 255).astype(np.uint8)


    # save the image
    fname = f'{name}_final.jpg'
    skio.imsave(fname, new_stacked)

    
    

In [106]:
image_pyramid("church")

b r displacement (-4, 58)
b g displacement (4, 26)


In [107]:
image_pyramid("emir")

b r displacement (56, 104)
b g displacement (24, 50)


In [108]:
image_pyramid("gundukush_dam")

b r displacement (-60, 132)
b g displacement (-30, 56)


In [109]:
image_pyramid("harvesters")

b r displacement (14, 124)
b g displacement (16, 60)


In [110]:
image_pyramid("icon")

b r displacement (22, 90)
b g displacement (16, 40)


In [111]:
image_pyramid("irrigation_canal")

b r displacement (48, 46)
b g displacement (24, 28)


In [112]:
image_pyramid("lady")

b r displacement (12, 118)
b g displacement (8, 54)


In [113]:
image_pyramid("lugano")

b r displacement (38, 86)
b g displacement (12, 36)


In [114]:
image_pyramid("melons")

b r displacement (14, 178)
b g displacement (10, 82)


In [115]:
image_pyramid("onion_church")

b r displacement (36, 108)
b g displacement (26, 52)


In [116]:
image_pyramid("sculpture")

b r displacement (-26, 140)
b g displacement (-10, 34)


In [117]:
image_pyramid("self_portrait")

b r displacement (36, 174)
b g displacement (30, 80)


In [118]:
image_pyramid("three_generations")

b r displacement (10, 112)
b g displacement (14, 54)


In [119]:
image_pyramid("train")

b r displacement (32, 88)
b g displacement (6, 42)
